In [ ]:
import os
import numpy as np
np.random.seed(10)
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import ttest_1samp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
from IPython.display import clear_output
import nibabel as nib

# Running Learning Analyses

1. [Helper functions](#helper-functions)
2. [Initial setup](#initial-setup) 
    * [Finding subjects](#finding-subjects)
3. [Learning analysis](#learning-analysis) 
    * [Setting parameters](#set-parameters)
    * [ROI effects](#main-effects)
    * [DG effect](#DG-effects)
    * [NMPH Model fitting](#nmph-model)
4. [Comparing to noise](#noise-dist) 
5. [Plotting results](#plotting) 

<a id='helper-functions'></a>
## Helper functions

In [ ]:
def subset(fullarray, run, AIX, BIX):
    critRun = fullarray[:,run]
    critRun = np.swapaxes(critRun, 0, -1)
    relCorrs = np.swapaxes(critRun[AIX, BIX], 0, -1)
    return relCorrs

def shufPrepost(fullarray):
    PAs = np.arange(0,15,2)
    PBs = np.arange(1,16,2)
    AIX = np.random.choice(PAs, 8, replace = False)
    BIX = np.random.choice(PBs, 8, replace = False)
    pre = subset(fullarray, 0, AIX, BIX)
    post = subset(fullarray, 7, AIX, BIX)
    return pre, post

def func(x, a, b, c, d):
    return a + b * x + c * x ** 2 + d * x ** 3

def lv1outPoly(DF, subject, constrain=False, plot=False, order=3):
    # Isolate single subject ...
    thisSub = DF[DF['subID'] == subject]
    # ... from the remaining subjects
    lv1out = DF[DF['subID'] != subject]
    # Assign as training (n - 1 subjects) and test (held out subject) data
    trainy = np.asarray(lv1out['Change'])
    trainx = np.asarray(pd.to_numeric(lv1out['simLevel']))
    testy = np.asarray(thisSub['Change'])
    testx = np.asarray(pd.to_numeric(thisSub['simLevel']))

    if constrain:
        weights, _ = curve_fit(func, trainx, trainy, bounds=([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf]))
        weights = weights[::-1]
    else: 
        # fit the specified order polynomial
        weights = np.polyfit(trainx, trainy, order)
    
    # Use the weights to build a model
    model = np.poly1d(weights)
    # Compute correlation between actual and predicted data
    if plot:
        plt.plot(np.arange(8), model(np.arange(8)))
    correspondence = np.corrcoef(np.vstack((testy, model(testx))))[0,1]
    return correspondence

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

<a id='initial-setup'></a>
## Initial setup

<a id='finding-subjects'></a>
#### Finding subjects
This cell lists the files in the current directory, and then subsets them to include only subjects relevant to the two relevant analyses.

In [ ]:
# Compile subjects
fullList = [file.split(".")[0] for file in os.listdir(".") if file.split(".")[-1] == "gz" and 'sub' in file]
fullList.sort()
subList = [i for i in fullList if i not in ["sub-23", "sub-38", "sub-39", "sub-40", "sub-41"]]
visList = fullList

print('{} subjects for learning analysis '.format(len(subList)))
print('{} subjects for visual analysis'.format(len(visList)))
print(subList)

<a id='learning-analysis'></a>
## Learning analysis

<a id='set-parameters'></a>
#### Setting initial parameters
* Choose the parameters in this first cell. First, establish whether you want to do this analysis in a single hemisphere or collapsed across both, or all three.
* Then, which ROIs you are interested in.

In [ ]:
# Which hemis to run the analysis in
runTypes = ['both']
# List the ROIs to conduct the analysis in
roiList = ['hipp', 'ca1', 'ca23', 'dg']  # ['hipp', 'ca1', 'ca23', 'dg', 'ca23dg', 'sub']
#roiList = ['PHC', 'perirhinal', 'EC']

These establish a dictionary to appropriately name files, provide a filename base to later fill depending on subject, ROI and hemishpere.

In [ ]:
masterROIs = ['hipp', 'ca1', 'ca23', 'dg', 'ca23dg', 'sub', 
              'phc', 'prc', 'ec', 'PHC', 'perirhinal', 'EC']
masterCOLs = [(218, 67, 68), (241,161,104), (78, 128, 130), 
              (79, 200, 120), (50,2,31), (10,36,99), 
              (13, 29, 118), (51, 130, 121), (91, 160, 74),
              (13, 29, 118), (51, 130, 121), (91, 160, 74)]
masterNOM = ['HC', 'CA1', 'CA2/3', 'DG', 'CA2/3/DG', 'SUB', 
             'PHC', 'PRC', 'EC', 'PHC', 'PRC', 'EC']
nameDict = dict(zip(masterROIs, masterNOM))
colorDict = dict(zip(masterROIs, masterCOLs))

In [ ]:
# List of types of analysis to do, total set is left hemi, right hemi, then combined across
types = ['left', 'right', 'both']
stems = ['_left', '_right', '']
stemdict = dict(zip(types, stems))

# Base file names for the two types of numpy arrays - these are filled later in the code
critBase = '{}/allsubs_allruns{}_{}.npy'
allBase = '{}/allims_runs{}_{}.npy'

<a id='main-effects'></a>
#### Computing the main effects in a given ROI
The following analyses find the representational change at each similarity level, quantifying the true effect in the sample, then shuffle AB pairings as many times as is dictated above and store these values.

This step reads in one of two numpy arrays, the first contains only the correlations for the relevant image pairings, while the other contains the entire 16 x 16 imagewise correlation matrix for every run.

In [ ]:
for hemi in runTypes:
    stem = stemdict[hemi]
    for i, roi in enumerate(roiList[:]):
        
        # This file is only the relevant correlations
        CritPairs = np.load(critBase.format('surf', stem, roi))
        print('Running {} hemisphere {}'.format(hemi, roi))
        
        # This computes the true effect in the whole sample.
        all_pre = CritPairs[:,0]
        all_post = CritPairs[:,7]
        thisDiff = all_post - all_pre
        
        # This saves the output to a csv file
        outFile = pd.DataFrame(thisDiff)
        outFile.to_csv('./csvout/{}_{}.csv'.format(roi, hemi))
print('Finished')

<a id='DG-effects'></a>
#### Unpacking the main effect in DG
The following quantifies the true effect at each similarity level, then the differences between peaks and trough.

How many resamplings?

In [ ]:
shuffles = 50000

In [ ]:
simDict = {}

us = []
for i in range(shuffles):
    samp = np.random.choice(36,36,True)
    _samp = np.mean(thisDiff[samp], 0)
    us.append(_samp)
us = np.array(us)
us = np.arctanh(us)
for simlevel in range(8):
    sim = us[:,simlevel]
    LB = np.percentile(sim, 2.5)
    U = np.mean(sim)
    UB = np.percentile(sim, 97.5)
    simDict[simlevel] = [LB, U, UB]

In [ ]:
for simlevel in range(8):
    LB, U, UB = simDict[simlevel]
    print('For similarity level {} -- M = {}, CI95 = [{} {}]'.format(simlevel + 1, 
                                                                     np.around(U, 4),
                                                                     np.around(LB, 4), 
                                                                     np.around(UB, 4)))

In [ ]:
for pairing in [[3,4], [7,4]]:
    sim1 = us[:,pairing[0]]
    sim2 = us[:,pairing[1]]
    diff = sim1-sim2
    LB = np.percentile(diff, 2.5)
    U = np.mean(diff)
    UB = np.percentile(diff, 97.5)
    print('For {} - {} contrast -- M = {}, CI95 = [{} {}]'.format(pairing[0]+1, pairing[1] + 1, 
                                                                  np.around(U, 17),
                                                                  np.around(LB, 4), 
                                                                  np.around(UB, 4)))

<a id='nmph-model'></a>
#### NMPH Model Fitting
This fits the cubic model predicted by the NMPH based on all but a held out subject, then predicts values for the held out subject based on the fit parameters from the remainder of the sample. What follows is a bootstrap resampled estimate of the average correlation between model predictions and and actual observed values.

Do you want to change which ROIs or hemispheres are analyzed? 

In [ ]:
roiList = ['hipp', 'ca1', 'ca23', 'dg']
roiList = ['PHC', 'perirhinal', 'EC']
runTypes = ['both']
resultDict = {}

How many resamplings?

In [ ]:
shuffles = 50000

Run the code ...

In [ ]:
for rn, roi in enumerate(roiList[:]):
    for hemi in runTypes:
        print('Running {} hemisphere {}'.format(hemi, roi))
        clear_output(wait=True)
        RESULTS = []
        AICS = []
        
        # read in the relevant data, subset to only the critical values
        dat = pd.read_csv('./csvout/{}_{}.csv'.format(roi, hemi))
        raw = np.array(dat.iloc[:,1:])
        
        # convert to pandas dataframe suitable for this analysis
        melted = pd.melt(dat, id_vars='Unnamed: 0', var_name='simLevel', value_name='Change')
        melted.rename(columns={'Unnamed: 0':'subID'}, inplace=True)
        rvals = []
        
        for subject in range(raw.shape[0]):
            # compute actual ~ predicted correlation for a given held out subject
            corr = lv1outPoly(melted, subject, constrain=True, plot=True, order=3)
            # Compile subjects in list
            rvals.append(corr)
        rvals = np.array(rvals)
        RVALS = rvals if rn == 0 else np.vstack((RVALS, rvals))
        print(RVALS.shape)
        us = np.zeros((shuffles))
        # Bootstrap resample a number of times, find 95% CI
        for shuf in range(shuffles):
            print('Running {} hemisphere {} -- {}/{}'.format(hemi, roi, shuf+1, shuffles))
            clear_output(wait=True)
            test = np.random.choice(rvals.shape[0], rvals.shape[0])
            u = np.mean(rvals[test])
            us[shuf] = u
        # Fisher transform for statistical analysis
        us = np.arctanh(us)
        LB = np.percentile(us, 2.5)
        U = np.mean(us)
        UB = np.percentile(us, 97.5)
        resultDict['{}_{}'.format(hemi, roi)] = [LB, U, UB]

        plt.xticks(np.arange(8), np.arange(1,9))
        plt.title('Predictions for Held-out Subjects')
        plt.xlabel('Similarity Level')
        plt.ylabel('Change in Representational Similarity')
        #plt.savefig('figures/predictions_{}_{}.pdf'.format(hemi, roi), dpi=600)
        plt.show()
#np.save("UShapeR.npy", RVALS)

<a id='nmph-model'></a>
#### Print bootstrap resampled results
The following cell will print the output of your chosen hemispheres and ROIs.

In [ ]:
for i, roi in enumerate(roiList):
    for hemi in runTypes:
        LB, U, UB = resultDict['{}_{}'.format(hemi, roi)]
        print('For {} hemisphere {} -- M = {}, CI95 = [{} {}]'.format(hemi, roi, 
                                                                      np.around(U, 4),
                                                                      np.around(LB, 4), 
                                                                      np.around(UB, 4)))

<a id='noise-dist'></a>
### Comparing to noise
Now, we explore how the distribution of values would look if the A and B items in a pair were arbitrary, rather than based on visual similarity, to ensure that the true effect in a given ROI exceeds the noise.

In [ ]:
noiseDict = {}
peaksDict = {}
shuffles = 5

In [ ]:
for hemi in runTypes:
    stem = stemdict[hemi]
    for i, roi in enumerate(roiList[:]):
        # This is every intercorrelation (16 x 16)
        AllIms = np.load(allBase.format('surf', stem, roi))
        us = np.zeros((shuffles))
        valleys = np.zeros((shuffles, 5))
        for shuf in range(shuffles):
            print('Running {} hemisphere {} -- {}/{}'.format(hemi, roi, shuf+1, shuffles))
            clear_output(wait=True)
           
            all_pre, all_post = shufPrepost(AllIms)  
            thisDiff = all_post - all_pre
            valleys[shuf, 0] = np.mean(thisDiff, 0)[4]
            valleys[shuf, 1] = np.mean(thisDiff, 0)[5]
            valleys[shuf, 2] = np.mean(thisDiff, 0)[7]
            valleys[shuf, 3] = np.mean(thisDiff, 0)[4] - np.mean(thisDiff, 0)[3]
            valleys[shuf, 4] = np.mean(thisDiff, 0)[7] - np.mean(thisDiff, 0)[4]
            dat = pd.DataFrame(thisDiff)
            dat['subID'] = np.arange(len(subList))
            thisIter = pd.melt(dat, id_vars=['subID'], var_name='simLevel', value_name='Change')
            rvals = []
            for subject in range(len(subList)):
                # compute actual ~ predicted correlation for a given held out subject
                corr = lv1outPoly(thisIter, subject, constrain=True, order=3)
                # Compile subjects in list
                rvals.append(corr)
            us[shuf] = np.mean(np.array(rvals))
        print(np.mean(valleys, 0))
        us = np.arctanh(us)
        _, trueU, _ = resultDict['{}_{}'.format(hemi, roi)]
        percentile = (us > trueU).sum() / shuffles
        LB = np.percentile(us, 2.5)
        U = np.mean(us)
        UB = np.percentile(us, 97.5)
        noiseDict['{}_{}'.format(hemi, roi)] = [LB, U, UB, percentile]
        plt.hist(us,100)
        plt.axvline(LB)
        plt.axvline(UB)
        #plt.savefig('figures/noise_{}_{}.pdf'.format(hemi, roi), dpi=600)
        plt.show()
print('Finished')

In [ ]:
plt.hist(us)

In [ ]:
for i, roi in enumerate(roiList):
    for hemi in runTypes:
        LB, U, UB, percentile = noiseDict['{}_{}'.format(hemi, roi)]
        print('For {} hemisphere {} -- M = {}, CI95 = [{} {}], percentile = {}'.format(hemi, roi, 
                                                                                       np.around(U, 4),
                                                                                       np.around(LB, 4), 
                                                                                       np.around(UB, 4),
                                                                                       np.around(percentile, 4)))

<a id='posthoc'></a>
### Post-hoc analyses of DG curve
This will plot the actual values, bootstrap resampled across subjects.

In [ ]:
descriptions = ['sim level 5', 'sim level 6', 'sim level 7']
for i, simlevel in enumerate([4,5,7]):
    noise = valleys[:, i]
    _, trueparam, _ = simDict[simlevel]
    percentile = (noise < trueparam).sum() / shuffles if i < 2 else (noise > trueparam).sum() / shuffles
    extremeNoise = np.around(np.amin(noise), 4) if i < 2 else np.around(np.amax(noise), 4)
    print('{} --- extreme noise value: {}, true value: {}, percentile WRT noise: {}'.format(descriptions[i],
                                                                                            extremeNoise,
                                                                                            np.around(trueparam,3),
                                                                                            percentile))
descriptions = ['peak1-trough diff', 'trough-peak2 diff']
for i, (truediff, noisediff) in enumerate(zip([0.12920655615343055, 0.15013934406029517], [3, 4])):
    noise = valleys[:, noisediff]
    percentile = (noise > truediff).sum() / shuffles
    extremeNoise = np.around(np.amax(noise), 4)
    print('{} --- extreme noise value: {}, true value: {}, percentile WRT noise: {}'.format(descriptions[i],
                                                                                            extremeNoise,
                                                                                            np.around(truediff,3),
                                                                                            percentile))

<a id='plotting'></a>
### Plotting the function
This will plot the actual values, bootstrap resampled across subjects.

Do you want to change which ROIs or hemispheres are analyzed? 

In [ ]:
roiList = ['hipp', 'ca1', 'ca23', 'dg']
runTypes = ['both']
plottingDict = {}

How many resamplings?

In [ ]:
shuffles = 50000

Run the code ...

In [ ]:
for i, roi in enumerate(roiList):
    for hemi in runTypes:
        dat = np.array(pd.read_csv('./csvout/{}_{}.csv'.format(roi, hemi)))[:, 1:]
        us = np.zeros((shuffles, 8))
        for shuf in range(shuffles):
            IX = np.random.choice(dat.shape[0], dat.shape[0])
            test = dat[IX, :]
            us[shuf] = (np.mean(test, 0))
        LB = np.percentile(us, 2.5, 0)
        U = np.mean(us, 0)
        UB = np.percentile(us, 97.5, 0)
        plottingDict['{}_{}'.format(hemi, roi)] = [LB, U, UB]

In [ ]:
for i, roi in enumerate(roiList):
    r,g,b = colorDict[roi]
    r, g, b = r/255, g/255, b/255
    for hemi in runTypes:
        LB, U, UB = plottingDict['{}_{}'.format(hemi, roi)]
        fig, ax = plt.subplots(figsize=(6,8))
        plt.axhline(0, zorder=-5, color=(0.5, 0.5, 0.5), lw=2, ls='--')
        plt.fill_between(np.arange(8), LB, UB, color = (1, 1, 1, 0.7), zorder=-2, lw=0)
        plt.fill_between(np.arange(8), LB, UB, color = (r, g, b, 0.1), zorder=-2, lw=0)
        plt.plot(U, lw=4, c=[r,g,b,1])
        plt.scatter(np.arange(8), U, s=150, lw=1.5, edgecolor=[[r,g,b,1]], facecolor=[[r,g,b,0.6]])
        plt.yticks(np.arange(-0.20, 0.16, 0.05), ['-0.20', '-0.15', '-0.10', '-0.05', '0.00', '0.05', '0.10', '0.15'], 
                   fontsize=18, **{'fontname':'Arial Narrow'})
        plt.xticks(np.arange(8), np.arange(1,9), fontsize=18, **{'fontname':'Arial Narrow'})
        plt.ylabel('Representational Change', fontsize=24, **{'fontname':'Arial Narrow'})
        plt.xlabel('Similarity Level', fontsize=24, **{'fontname':'Arial Narrow'})
        #plt.savefig('figures/truedata_{}_{}.pdf'.format(hemi, roi), dpi=600)
        plt.show()

#### Figure for Paper

This is the figure that ends up in the paper, and depicts the learning effect in the subfields and hippocampus as a whole.

In [ ]:
roiList = ['hipp', 'ca1', 'ca23', 'dg']
sig = ['','','','*']
sigdict = dict(zip(roiList, sig))

In [ ]:
fig, axes = plt.subplots(1, len(roiList), figsize=(int(len(roiList)*4), 8), sharey=True)
for hemi in runTypes:
    for i, (ax, roi) in enumerate(zip(axes, roiList)):
        r,g,b = colorDict[roi]
        r, g, b = r/255, g/255, b/255
        LB, U, UB = plottingDict['{}_{}'.format(hemi, roi)]
        ax.axhline(0, zorder=-5, color=(0.5, 0.5, 0.5), lw=2, ls='--')
        ax.fill_between(np.arange(8), LB, UB, color = (1, 1, 1, 0.7), zorder=-2, lw=0)
        ax.fill_between(np.arange(8), LB, UB, color = (r, g, b, 0.1), zorder=-2, lw=0)
        ax.plot(U, lw=4, c=[r,g,b,1])
        ax.scatter(np.arange(8), U, s=150, lw=1.5, edgecolor=[[r,g,b,1]], facecolor=[[r,g,b,0.6]])
        ax.set_xticks(np.arange(8))
        ax.set_xticklabels(np.arange(1,9), fontsize=20, **{'fontname':'Arial Narrow'})
        ax.set_xlabel('Similarity Level', fontsize=28, **{'fontname':'Arial Narrow'})
        ax.text(0, -0.18, '{}{}'.format(nameDict[roi],sigdict[roi]), color=(r,g,b,1), fontsize=64, **{'fontname':'DIN Condensed'})
        if i == 0:
            ax.set_ylabel('Representational Change', fontsize=28, **{'fontname':'Arial Narrow'})
            ax.set_yticks(np.arange(-0.20, 0.16, 0.05))
            ax.set_yticklabels(['-0.20', '-0.15', '-0.10', '-0.05', '0.00', '0.05', '0.10', '0.15'],
                               fontsize=20, **{'fontname':'Arial Narrow'})
    plt.ylim(-0.20, 0.15)
    plt.tight_layout()
    plt.savefig('figures/manuscript_{}.pdf'.format(hemi), dpi=600)
    #plt.savefig('figures/supplement_{}.pdf'.format(hemi), dpi=600)
    plt.show()

#### Brains for ROI view

This is the slice that shows the hippocampal subfield segmentations

In [ ]:
anat = 'brains/sub10_t2int1.nii.gz'
subStem = 'brains/{}.nii.gz'
cmap = plt.get_cmap('gist_gray')
new_cmap = truncate_colormap(cmap, 0.3, 1.0)

base = nib.load(anat).get_data()

for i, roi in enumerate(roiList):
    fig, ax = plt.subplots(1,1, figsize=(12,4))
    r,g,b = colorDict[roi]
    r, g, b = r/255, g/255, b/255
    mask = nib.load(subStem.format(roi)).get_data()
    forNow = np.copy(base)
    forNow[mask == 1] = np.nan
    thisSlice = np.rot90(forNow[45:165, 131,85:120])
    ax.matshow(thisSlice, cmap=new_cmap)
    for Y in range(thisSlice.shape[0]):
        for X in range(thisSlice.shape[1]):
            if np.isnan(thisSlice[Y, X]):
                rect = patches.Rectangle((X-0.6, Y-0.6),1.1,1.1,linewidth=0,edgecolor='r',facecolor=(r,g,b))
                ax.add_patch(rect)
    
    plt.xticks([])
    plt.yticks([])
    plt.axis('off')
    plt.savefig('figures/brain_{}.pdf'.format(roi), dpi=600)
    plt.show()
    
    